# 1. Import libraries, data, and globals

In [1]:
%matplotlib inline

# Regular libraries
import requests
import pandas as pd
import string
import time
from bs4 import BeautifulSoup
from pandas.plotting import scatter_matrix
import os 
import string
import random
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Get rid of warnings
import warnings
warnings.filterwarnings('ignore')

# Sklearn Specific
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import cross_validate

# sklearn algos
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB # Naive Bayes
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier # SGD
from sklearn.neural_network import MLPClassifier # neural network (multilayer perceptron)

fs_df = pd.read_csv("../data/full_stats_v2.csv")
fc_df = pd.read_csv("../data/fighter_char.csv")

/anaconda3/envs/fight_prediction/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


# 2. Data wrangling

#### Add Totals

In [2]:
fs_df = fs_df.fillna(0) # Fill NaN with 0

# Calc time of fight in seconds
fs_df['duration'] = fs_df.apply(lambda x: (int(x['end_round'])-1)*60*5 + int(x['end_time'].split(":")[0])*60 + int(x['end_time'].split(":")[1]) , axis = 1)

# Calc totals for each fighter
which_fighter = ["f1","f2"]
stat_name_ending = ["kds","sigstrikes_l","sigstrikes_a","strikes_l","strikes_a","tds_l","tds_a","subs_a","passes",
                   "revs","ss_head_l","ss_head_a","ss_body_l","ss_body_a","ss_leg_l","ss_leg_a","ss_dist_l","ss_dist_a",
                   "ss_clinch_l","ss_clinch_a","ss_ground_l","ss_ground_a"]

for f in which_fighter:
    
    for stat in stat_name_ending:
        
        fs_df[f+'_t_' + stat] = np.sum(fs_df.loc[:,(fs_df.columns.str.startswith(f) & fs_df.columns.str.endswith(stat))], axis =1)
        
        fs_df[f+'_pm_' + stat] =  fs_df[f+'_t_' + stat] / fs_df['duration'] * 60


#### Subset the Data for Key Stats and Per Minute 

In [3]:
matchers = ['fighter','winner',"_pm_","date"]
ml_cols = [s for s in fs_df.columns if any(xs in s for xs in matchers)]
ml_df = fs_df[ml_cols]

#### Make each fight have two rows, so that the label is win or lose

In [4]:
# Part 1
part_1_df = ml_df.copy()

part1_key_cols = part_1_df.columns
part1_value_cols = []

for col in part1_key_cols:
    new_col = col.replace("f1","f")
    new_col = new_col.replace("f2","o")
    new_col = new_col.replace("fighter_1","a_fighter")
    new_col = new_col.replace("fighter_2","a_opponent")
    new_col = new_col.replace("winner","a_winner")
    part1_value_cols.append(new_col)
    
p1_col_dict = dict(zip(part1_key_cols, part1_value_cols))

part_1_df = part_1_df.rename(columns = p1_col_dict)

In [5]:
# Part 2
part_2_df = ml_df.copy()

part2_key_cols = part_2_df.columns
part2_value_cols = []

for col in part2_key_cols:
    new_col = col.replace("f1","o")
    new_col = new_col.replace("f2","f")
    new_col = new_col.replace("fighter_1","a_opponent")
    new_col = new_col.replace("fighter_2","a_fighter")
    new_col = new_col.replace("winner","a_winner")
    part2_value_cols.append(new_col)
    
p2_col_dict = dict(zip(part2_key_cols, part2_value_cols))

part_2_df = part_2_df.rename(columns = p2_col_dict)

In [6]:
two_pf = pd.concat([part_1_df, part_2_df], sort = True)
two_pf = two_pf.sort_values(by ='date', ascending= False)
two_pf['result'] = two_pf.apply(lambda x: int(x['a_fighter'] == x['a_winner']), axis =1)

#### Show head and describe the wrangled dataset

In [7]:
two_pf.head(3)

,a_fighter,a_opponent,a_winner,date,f_pm_kds,f_pm_passes,f_pm_revs,f_pm_sigstrikes_a,f_pm_sigstrikes_l,f_pm_ss_body_a,...,o_pm_ss_head_a,o_pm_ss_head_l,o_pm_ss_leg_a,o_pm_ss_leg_l,o_pm_strikes_a,o_pm_strikes_l,o_pm_subs_a,o_pm_tds_a,o_pm_tds_l,result
0,"Israel Adesanya ""The Last Stylebender""","Anderson Silva ""The Spider""","Israel Adesanya ""The Last Stylebender""",2019-02-09,0.0,0.000000,0.000000,11.733333,5.666667,0.933333,...,5.266667,1.800000,2.066667,1.933333,14.920000,6.537778,0.000000,0.0,0.000000,1
1,"Lando Vannata ""Groovy""","Marcos Mariano ""Dhalsim""","Lando Vannata ""Groovy""",2019-02-09,0.0,0.813559,0.000000,16.271186,8.135593,0.203390,...,7.525424,3.661017,0.406780,0.406780,6.270612,2.158001,0.000000,0.0,0.000000,1
10,"Jonathan Martinez ""Dragon""","Wuliji Buren ""Beast Master""","Wuliji Buren ""Beast Master""",2019-02-09,0.0,0.266667,0.066667,5.800000,3.133333,0.533333,...,4.266667,2.066667,1.600000,1.066667,9.657778,5.164444,0.066667,0.6,0.266667,0


In [8]:
two_pf.describe()

,f_pm_kds,f_pm_passes,f_pm_revs,f_pm_sigstrikes_a,f_pm_sigstrikes_l,f_pm_ss_body_a,f_pm_ss_body_l,f_pm_ss_clinch_a,f_pm_ss_clinch_l,f_pm_ss_dist_a,...,o_pm_ss_head_a,o_pm_ss_head_l,o_pm_ss_leg_a,o_pm_ss_leg_l,o_pm_strikes_a,o_pm_strikes_l,o_pm_subs_a,o_pm_tds_a,o_pm_tds_l,result
count,7434.000000,7434.000000,7434.000000,7434.000000,7434.000000,7434.000000,7434.000000,7434.000000,7434.000000,7434.000000,...,7434.000000,7434.000000,7434.000000,7434.000000,7434.000000,7434.000000,7434.000000,7434.000000,7434.000000,7434.000000
mean,0.109153,0.155623,0.015604,10.523933,4.787612,1.264215,0.861380,1.071830,0.736254,8.085448,...,8.346284,3.209708,0.913661,0.718150,37.801289,18.426116,0.079519,0.356855,0.137669,0.490987
std,0.512414,0.389497,0.117449,8.857232,5.388737,1.149576,0.889993,2.052800,1.431561,6.348670,...,6.911298,3.761260,0.995655,0.805858,42.793170,28.338528,0.301374,0.510187,0.277029,0.499952
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,5.400000,2.133333,0.524017,0.322725,0.133333,0.066667,3.666667,...,4.066667,1.266667,0.230843,0.166493,19.299867,8.239358,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,8.517699,3.582090,1.000000,0.666667,0.533927,0.335196,6.666667,...,6.666667,2.233215,0.659341,0.500000,28.897778,13.249778,0.000000,0.200000,0.000000,0.000000
75%,0.000000,0.144928,0.000000,13.020113,5.666667,1.666667,1.133333,1.266667,0.866667,10.866667,...,10.500000,3.800000,1.266667,1.000000,43.382521,20.069622,0.000000,0.527705,0.200000,1.000000
max,9.230769,9.600000,8.571429,137.142857,112.000000,21.000000,18.000000,51.176471,35.294118,75.000000,...,94.285714,56.250000,22.000000,16.000000,1016.326531,792.000000,8.571429,7.500000,7.500000,1.000000


# 3. Create cumulative statistics data with specified look back window

#### Create functions to make the look back window dataframe

In [9]:
def generate_fighters_dict():
    cols = two_pf.columns.tolist()
    fighters = {}
    for index, row in two_pf.iterrows():
        fighter = row["a_fighter"]
        if fighter not in fighters:
            fighters[fighter] = [row]
        else:
            fighters[fighter].append(row)
    for fighter in fighters:
        rows_list = fighters[fighter]
        fighters[fighter] = pd.DataFrame(columns=cols, data=rows_list)
    return fighters

fighters_dict = generate_fighters_dict() # Generates a dictionary with each fighter having his own pd.DataFrame()

In [11]:
def get_fighter_info(past_df, is_fighter, lb_num):
    latest_df = past_df.head(lb_num)
    latest_lb_cum = latest_df.mean(skipna=True, numeric_only=True).to_dict()
    new_dict = {}
    if is_fighter:
        for var in latest_lb_cum:
            if var[0] == "f":
                new_dict[var] = latest_lb_cum[var]
    else:
        for var in latest_lb_cum:
            if var[0] == "f":
                new_var = var.replace("f_", "o_")
                new_dict[new_var] = latest_lb_cum[var]
    return(new_dict)

In [12]:
def generate_cum_df(fighters_dict, lb_num):
    
    cumulative_df = pd.DataFrame(columns = two_pf.columns)
    
    for index, row in two_pf.iterrows():
        fighter_name = row["a_fighter"]
        opponent_name = row["a_opponent"]
        fight_date = row["date"]
        
        fighter_df = fighters_dict[fighter_name]
        opponent_df = fighters_dict[opponent_name]
        
        past_fighter_df = fighter_df.loc[fighter_df['date'] < fight_date]
        if past_fighter_df.shape[0] < lb_num:
            continue
        past_opponent_df = opponent_df.loc[opponent_df['date'] < fight_date]
        if past_opponent_df.shape[0] < lb_num:
            continue
        fighter_info = get_fighter_info(past_fighter_df, True, lb_num)
        opponent_info = get_fighter_info(past_opponent_df, False, lb_num)
        all_info = dict(fighter_info, **opponent_info)
        all_info['a_fighter'] = row['a_fighter']
        all_info['a_opponent'] = row['a_opponent']
        all_info['a_winner'] = row['a_winner']
        all_info['date'] = row['date']
        all_info['result'] = row['result']
        
        all_info_df = pd.DataFrame(all_info, index=[0])
        
        cumulative_df = pd.concat([cumulative_df, all_info_df])
        
    return(cumulative_df)
        

#### Make cumulative data

In [34]:
cumu_1_df = generate_cum_df(fighters_dict,1)    
cumu_3_df = generate_cum_df(fighters_dict,3)    
cumu_5_df = generate_cum_df(fighters_dict,5)
cumu_10_df = generate_cum_df(fighters_dict,10)

In [35]:
cumu_dfs_dict = {"Cumulative Data: 1 Fight Lookback Window": cumu_1_df,
                 "Cumulative Data: 3 Fight Lookback Window": cumu_3_df,
                 "Cumulative Data: 5 Fight Lookback Window": cumu_5_df,
                "Cumulative Data: 10 Fight Lookback Window": cumu_10_df} #  create dictionary of the dfs

# 4. Generate machine learning models

#### Initialize classifiers

In [36]:
models = []

blr_clf = LogisticRegression()
rf_clf = RandomForestClassifier()
svm_clf = svm.SVC()
knn_clf = KNeighborsClassifier()
dtree_clf = DecisionTreeClassifier()
nb_clf = GaussianNB()
perc_clf = Perceptron()
sgd_clf = SGDClassifier()
mlp_clf = MLPClassifier()

models.extend((blr_clf ,rf_clf, svm_clf, knn_clf, dtree_clf, nb_clf, perc_clf, sgd_clf,mlp_clf))



In [37]:
# 5. Train and print scores for each model

In [41]:
for key in cumu_dfs_dict:
    df = cumu_dfs_dict[key]
    print(key + " - Number of observations ="+ str(df.shape[0])+"\n")
    # Split features and labels
    X = df.iloc[:,4:-1]
    y = df['result'].astype('int')

    for model in models:
        print(" " + type(model).__name__)
        
       # model.fit(X, y)
        
        scoring_stats = {'accuracy': 'accuracy',
               'recall': 'recall',
               'precision': 'precision',
               'roc_auc': 'roc_auc'}
        
        avg_scores = cross_validate(model, X, y, cv=3, scoring = scoring_stats)
    
        print("     Accuracy: " + str(np.mean(avg_scores['test_accuracy'])))
        print("     Recall:" + str(np.mean(avg_scores['test_recall'])))
        print("     Precision:" + str(np.mean(avg_scores['test_precision'])))
        print("     ROC - AUC:" + str(np.mean(avg_scores['test_roc_auc'])))
        print("\n")

Cumulative Data: 1 Fight Lookback Window - Number of observations =5306

 LogisticRegression
     Accuracy: 0.5152674432385469
     Recall:0.4461066359800537
     Precision:0.5076628352490421
     ROC - AUC:0.5131916609100822


 RandomForestClassifier
     Accuracy: 0.5003748357628915
     Recall:0.3751438434982739
     Precision:0.48958371202481793
     ROC - AUC:0.5051135688656986


 SVC
     Accuracy: 0.5069737357046571
     Recall:0.37015726889144607
     Precision:0.49764725531717585
     ROC - AUC:0.5043696908634744


 KNeighborsClassifier
     Accuracy: 0.48869503012754434
     Recall:0.47295742232451093
     Precision:0.4792842408807805
     ROC - AUC:0.49019752041851145


 DecisionTreeClassifier
     Accuracy: 0.4979275856100583
     Recall:0.48369773686229384
     Precision:0.4889466233867399
     ROC - AUC:0.4976841586713315


 GaussianNB
     Accuracy: 0.5043354972984541
     Recall:0.39087073264288447
     Precision:0.49484499722888015
     ROC - AUC:0.5001665081087365


 

# Preliminary Analysis
- The hyperparameters from the sklearn classifiers were set to their defaults and will be tuned. 


- Nevertheless, the effectiveness of each model appears to vary based off the length of the look back number. I will soon be transforming the printed text data above into nicer looking graphs.  


- Please scroll around in the above cell to view the accuracy, recall, precision, and ROC-AUC from using a cross validate method. 


- Now, most of the accuracy percentages are hovering near the 50% mark. As noted by previous literature, this data is inherently noisy and will likely make it very difficult to have an accuracy of over 60%. It is even more of an issue when generating the data using a look back window because the number of observations decreases substantially. 



# (WIP) Making Cumulative Data

- Add the fighter's age, height, and reach has already been scraped and will be merged into the cumulative data by fighter names.


- Standardize/normalize fight data


- Perhaps use more data from before 2010. 